# Model 3

## Model: K-Nearest Neighbors <br>
### Considerations:
- The original data set was filtered to contemplate only two possible outcomes in the column `koi_disposition`: CONFIRMED and FALSE POSITIVE
- The column `koi_tce_plnt_num` was dropped from the initial data set. <br>
 
### Key takeaways:
- Best fit at {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}   best_score_: 0.99220915900256

### Resources:
- G, Erik. k-Neighbors Classifier with GridSearchCV Basics (2018). Link: https://medium.com/@erikgreenj/k-neighbors-classifier-with-gridsearchcv-basics-3c445ddeb657


In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
# Filter candidate features to work with confirmed and false positives
df = df[df["koi_disposition"] != "CANDIDATE"]
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
# Get dummy values from categorical variables
data_binary_encoded = pd.get_dummies(df, columns=["koi_disposition"])
data_binary_encoded = data_binary_encoded.drop(columns=["koi_disposition_FALSE POSITIVE"])
data_binary_encoded.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,koi_disposition_CONFIRMED
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,1
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,0
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,0
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,1
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714,1


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = data_binary_encoded.drop(columns=['koi_tce_plnt_num'])

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
y = selected_features["koi_disposition_CONFIRMED"]
X = selected_features.drop(columns="koi_disposition_CONFIRMED")

feature_names = X.columns
target_names = ["FALSE POSITIVE", "CONFIRMED"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)


In [7]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
979,0,1,1,1,41.077297,1.985000e-04,-1.985000e-04,379.128480,0.003110,-0.003110,...,-189,4.381,0.195,-0.214,0.938,0.263,-0.197,289.07703,51.761822,14.907
6201,0,1,0,0,13.639480,1.970000e-06,-1.970000e-06,144.531439,0.000123,-0.000123,...,-164,4.517,0.060,-0.140,0.846,0.184,-0.085,292.21732,49.707901,15.852
5648,0,1,0,0,14.854416,7.890000e-07,-7.890000e-07,136.559783,0.000044,-0.000044,...,-197,4.444,0.070,-0.224,1.022,0.350,-0.117,282.42578,43.884331,14.558
229,0,0,0,0,10.681695,5.170000e-05,-5.170000e-05,131.550820,0.003720,-0.003720,...,0,4.221,0.188,-0.101,1.250,0.196,-0.240,292.58044,46.097389,15.065
1302,0,0,0,0,12.509458,7.470000e-05,-7.470000e-05,136.992170,0.004600,-0.004600,...,-119,4.303,0.120,-0.108,1.184,0.183,-0.165,295.42200,38.735691,13.860


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_scaled, y_train)

print(f"Training Data Score: {knn.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {knn.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9811463046757164


In [11]:
from sklearn.metrics import classification_report
predictions = knn.predict(X_test_scaled)
print(classification_report(y_test, predictions, target_names = target_names))

precision    recall  f1-score   support

FALSE POSITIVE       0.98      0.99      0.99       875
     CONFIRMED       0.98      0.96      0.97       451

      accuracy                           0.98      1326
     macro avg       0.98      0.98      0.98      1326
  weighted avg       0.98      0.98      0.98      1326



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [12]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': [1, 3, 5, 11, 19],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']}

model = KNeighborsClassifier()
grid = GridSearchCV(model, param_grid, verbose=3)

In [13]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] metric=euclidean, n_neighbors=1, weights=uniform ................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  metric=euclidean, n_neighbors=1, weights=uniform, score=0.985, total=   1.0s
[CV] metric=euclidean, n_neighbors=1, weights=uniform ................
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[CV]  metric=euclidean, n_neighbors=1, weights=uniform, score=0.984, total=   0.7s
[CV] metric=euclidean, n_neighbors=1, weights=uniform ................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s
[CV]  metric=euclidean, n_neighbors=1, weights=uniform, score=0.987, total=   0.4s
[CV] metric=euclidean, n_neighbors=1, weights=uniform ................
[CV]  metric=euclidean, n_neighbors=1, weights=uniform, score=0.990, total=   0.4s
[CV] metric=euclidean, n_neighbors=1, weights=uniform ................
[CV]  metr

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [1, 3, 5, 11, 19],
                         'weights': ['uniform', 'distance']},
             verbose=3)

In [14]:
print(grid.best_params_)
print(grid.best_score_)

{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
0.99220915900256


In [15]:
predictions = grid.predict(X_test_scaled)
print(classification_report(y_test, predictions, target_names=target_names))

precision    recall  f1-score   support

FALSE POSITIVE       0.98      1.00      0.99       875
     CONFIRMED       1.00      0.96      0.98       451

      accuracy                           0.99      1326
     macro avg       0.99      0.98      0.98      1326
  weighted avg       0.99      0.99      0.99      1326



In [16]:
# Fit the hypertuned model to scaled data and get training and testing scores
knn_tuned = KNeighborsClassifier(metric='manhattan', n_neighbors=5, weights='distance')
knn_tuned.fit(X_train_scaled, y_train)

print(f"Training Data Score: {knn_tuned.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {knn_tuned.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9864253393665159


# Save the Model

In [17]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'final_model.sav'
joblib.dump(knn_tuned, filename)

['final_model.sav']